In [169]:
from pydantic import BaseModel, Field
from typing import TypedDict,Annotated,Union,Dict
from langchain_ollama import ChatOllama
from langgraph.graph import START,StateGraph,END,MessagesState
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AnyMessage,BaseMessage
from langchain_core.runnables import RunnableConfig
from langchain_groq import ChatGroq

import json
model = "jacob-ebey/phi4-tools:latest"
model2 = "qwen2.5:32b"
llm = ChatOllama(model=model,base_url="192.168.23.138:11439")
# llmReason = ChatGroq(api_key="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",model="deepseek-r1-distill-llama-70b")
# llmReason = ChatOllama(model="deepseek-r1:32b")

In [170]:

    
class Result(BaseModel):
    Critica_error : int = Field(...,ge=0,description="read the above response and extract the critical error")
    Critical_error_Reason : str = Field(...,description="read the above response and extract why the above response suggested the reason for critical error in about 2 lines")
    Inaccuracy_error: int = Field(...,ge=0,description="read the above response and extract the inaccuracy error")
    Inaccuracy_error_Reason: str = Field(...,description="read the above response and extract why the above response suggested the reason for inaccuracy error in about 2 lines")
    Total_no_experiment_steps: int = Field(...,ge=0,description="read the above response and extract the total number of experiment steps")
structed_llm=llm.with_structured_output(Result)

class State(TypedDict):
    # messages : Annotated[list[BaseMessage], add_messages]
    CustomSystemPrompt : str
    GroundTruth : str
    GeneratedExperiment : str
    evaluation: str
    evaluationOutput : str
    FinalResult : Result


In [171]:
def SystemPromptCreator(state: State, config: RunnableConfig):
    query = '''
    You are an experiment analyst. Your task is to evaluate other experiments by comparing them against the Ground Truth provided above. 
    Identify the key steps, conditions, and requirements that must be present for an experiment to be accurate and valid. 
    Using these key points, create a detailed system prompt that will help you consistently judge other experiments based on this Ground Truth.
    '''
    return {'CustomSystemPrompt': llm.invoke(state['GroundTruth'] + query).content}


def Evaluator(state: State, config: RunnableConfig):
    query = '''\n
    Compare the generated experiment with the Ground Truth provided above, step by step. 
    For each experiment step:
    1. Indicate whether the step is present in both, only in the Ground Truth, or only in the generated experiment.
    2. If present in both, compare the details and identify any differences in execution, measurements, materials, or procedures.
    3. Clearly explain the impact of each difference on the experiment's accuracy, validity, or outcome.
    4. Highlight any missing steps in the generated experiment that are critical according to the Ground Truth.
    5. Conversely, identify any additional steps in the generated experiment that are not present in the Ground Truth and discuss their significance.
    
    Provide a detailed breakdown for each experiment step, ensuring a comprehensive comparison between the Ground Truth and the generated experiment.
    The experiment to analyze, in relation to the Ground Truth, is: 
    '''

    return {'evaluation': llm.invoke(state['CustomSystemPrompt'] + query + state['GeneratedExperiment']).content}


def ScoreEvaluation(state: State, config: RunnableConfig):
    query = '''
    Evaluate the generated experiment by comparing it with the Ground Truth using the following criteria:
    
    1. Critical Error: Identify and count the number of steps or parts with critical errors that compromise the experiment's validity. 
    2. Inaccuracy Error: Count the number of inaccuracies that, while incorrect, do not severely affect the experiment's outcome (e.g., optional safety equipment). 
    3. Correct Step: Count how many parts of the experiment are accurate and align with the Ground Truth.
    4. Total Experiment Steps: Calculate the total number of steps present in the generated experiment.
    
    Provide a detailed breakdown of each category, clearly justifying the reasoning behind each classification.
    '''
    evaluation_output = llm.invoke(state['evaluation'] + query + state['GeneratedExperiment']).content
    return {'evaluationOutput': evaluation_output}


def ScoreEvaluationStructured(state: State, config=RunnableConfig):
    query = f'''
    Format the evaluation output to provide a clear and organized summary. Extract the numbers and present them in the following structured format:
    
    1. Critical Errors: (Number of Critical Errors)
    2. Critical Error Reason: (Summary of reasons for Critical Errors)
    3. Inaccuracy Errors: (Number of Inaccuracy Errors)
    4. Inaccuracy Error Reason: (Summary of reasons for Inaccuracy Errors)
    5. Correct Steps: (Number of Correct Steps)
    6. Total Number of Experiment Steps: (Total steps in the generated experiment)
    
    Ensure the output is well-organized and concise.
    '''
    return {'FinalResult': structed_llm.invoke(query + state['evaluationOutput'])}


In [172]:
build = StateGraph(State)
build.add_node("prompt",SystemPromptCreator)
build.add_node("evaluator",Evaluator)
build.add_node("score",ScoreEvaluation)
build.add_node('structuredResponse',ScoreEvaluationStructured)
build.add_edge(START,"prompt")
build.add_edge("prompt","evaluator")
build.add_edge("evaluator","score")
build.add_edge("score","structuredResponse")
build.add_edge("structuredResponse",END)
graph = build.compile()

In [173]:

gt = {
    "experiments": [
        {
            "title": "To set up and test a unity gain amplifier using an IC 741 operational amplifier",
            "objective": "To set up and test a unity gain amplifier using an IC 741 operational amplifier",
            "materials": [
                "Operational Amplifier IC (LM741)",
                "Variable Resistor (10KΩ)",
                "Resistor (1KΩ)",
                "Input Signal Source (Function Generator)",
                "Connecting Wires",
                "Power Supply (9V)"
            ],
            "procedure": {
                "power_supply_connections": [
                    "Connect the positive terminal of the battery to pin 7 (V+) of the IC 741",
                    "Connect the negative terminal of the battery to pin 4 (V-) of the IC 741"
                ],
                "input_signal_setup": [
                    "Attach one end of the potentiometer to the positive supply voltage",
                    "Connect the wiper (middle terminal) of the potentiometer to pin 3 (non-inverting input) of the op-amp",
                    "Connect the other end of the potentiometer to ground"
                ],
                "feedback_loop": [
                    "Connect a wire from pin 6 (output) directly to pin 2 (inverting input)"
                ],
                "load_resistor_connection": [
                    "Attach one terminal of the 1 kΩ resistor to pin 6 (output)",
                    "Connect the other terminal of the resistor to ground"
                ],
                "biasing_and_stability": [
                    "Ensure pins 4 (V-) and 7 (V+) are properly connected for stable operation",
                    "If necessary, connect a capacitor across V+ and V- for power supply decoupling"
                ],
                "testing_the_circuit": [
                    "Turn on the power supply and adjust the potentiometer to vary the input voltage",
                    "Measure the output voltage at pin 6 with a voltmeter",
                    "Verify that the output voltage matches the input voltage"
                ]
            },
            "analysis": [
                "Verify that the output voltage matches the input voltage, confirming the unity gain configuration",
                "Discuss the importance of negative feedback in achieving unity gain",
                "Analyze the effect of the variable resistor on the input voltage and output voltage",
                "Examine the role of the load resistor in the circuit and its impact on the output voltage"
            ]
        }
    ]
}

generated_exp = {
    "experiments": [
        {
            "title": "To set up and test a unity gain amplifier using an IC 741 operational amplifier",
            "objective": "Creating a voltage follower (also known as a buffer amplifier) using an operational amplifier like the 741 involves designing and setting up an experiment to test its performance.",
            "materials": [
                "Op-Amp IC (741 operational amplifier)",
                "Breadboard or PCB",
                "Dual Power Supply (+15V and -15V)",
                "Signal Generator",
                "Oscilloscope/Function Analyzer",
                "Connecting Wires"
            ],
            "procedure": {
                "understanding_the_circuit": [
                    "A voltage follower configuration connects the output of the op-amp directly to its inverting input (-), while the non-inverting input (+) receives the input signal.",
                    "The goal is for the output to follow (buffer) the input without any amplification or attenuation."
                ],
                "designing_the_circuit": [
                    "Connect pin 4 of the 741 op-amp to Vcc (+15V).",
                    "Connect pin 11 to Vee (-15V).",
                    "Connect pin 7 (Output) directly to pin 6 (Inverting Input, -).",
                    "Connect the input signal to pin 3 (Non-Inverting Input, +)."
                ],
                "setting_up_the_experiment": [
                    "Ensure that the dual power supply is correctly connected to the op-amp.",
                    "Set up the function generator to provide a sinusoidal or square wave signal within the common operating range of the op-amp.",
                    "Assemble the circuit on the breadboard, carefully connecting the input, output, and power supply as described."
                ],
                "testing_the_circuit": [
                    "Start with a low amplitude signal to avoid saturation (e.g., a few volts peak-to-peak).",
                    "Use an oscilloscope to observe both the input and output signals.",
                    "Verify that the output waveform matches the input waveform in shape, frequency, and phase.",
                    "Measure the voltage levels at the input and output to confirm they are identical (or very close), indicating a unity gain."
                ],
                "analysis": [
                    "Note any discrepancies between the input and output signals.",
                    "Assess bandwidth limitations by gradually increasing the signal frequency and noting where distortion or deviation occurs.",
                    "Consider potential issues like offset voltage or bias currents that may affect performance, especially at lower frequencies."
                ],
                "troubleshooting": [
                    "Check for correct power supply voltages.",
                    "Ensure all connections are secure and correct.",
                    "Verify the op-amp is not overheating; if it is, check for short circuits or excessive input signal amplitude."
                ],
                "documentation": [
                    "Record your observations, including any discrepancies between expected and actual performance.",
                    "Note conditions under which the voltage follower performs optimally."
                ]
            }
        }
    ]
}

In [ ]:
samples = []
for _ in range(10):
    input = {"CustomSystemPrompt":"", "GroundTruth":json.dumps(gt), "GeneratedExperiment":json.dumps(generated_exp), "output":"","evaluationOutput":"","FinalResult":{}}
    samples.append(dict(graph.invoke(input)['FinalResult']))
    print("done")
# output['output']


done
done
done
done
done
done
done


In [ ]:
samples

[{'Critica_error': 2,
  'Critical_error_Reason': 'Incorrect pin connections can lead to improper circuit functionality or damage to the op-amp. Different power supply configurations could affect operational range and stability, leading to incorrect performance analysis.',
  'Inaccuracy_error': 2,
  'Inaccuracy_error_Reason': "Using an external signal generator instead of a potentiometer affects flexibility but not fundamental outcomes. Omission of a load resistor could affect performance analysis under real-world loads, though it doesn't severely compromise validity.",
  'Total_no_experiment_steps': 8},
 {'Critica_error': 0,
  'Critical_error_Reason': 'No critical errors were identified as all deviations did not compromise safety or validity.',
  'Inaccuracy_error': 2,
  'Inaccuracy_error_Reason': "The inclusion of an oscilloscope and dual power supply deviates from the ground truth's simpler setup. Omission of a potentiometer for variable input voltage control limits flexibility but d

In [ ]:
import json
with open("evaluationTest2.json","w") as f:
    json.dump(samples,f,indent=4)

In [ ]:

inputs = {"messages": [("user", qurey)]}
graph.invoke({"messages":qurey,"CustomSystemPrompt":""})
# print(graph.state["CustomSystemPrompt"])

{'messages': [HumanMessage(content='{ "experiments": [ { "title": "To set up and test a unity gain amplifier using an IC 741 operational amplifier", "objective": "To set up and test a unity gain amplifier using an IC 741 operational amplifier", "materials": [ "Operational Amplifier IC (LM741)", "Variable Resistor (10KΩ)", "Resistor (1KΩ)", "Input Signal Source (Function Generator)", "Connecting Wires", "Power Supply (9V)" ], "procedure": { "power_supply_connections": [ "Connect the positive terminal of the battery to pin 7 (V+) of the IC 741", "Connect the negative terminal of the battery to pin 4 (V-) of the IC 741" ], "input_signal_setup": [ "Attach one end of the potentiometer to the positive supply voltage", "Connect the wiper (middle terminal) of the potentiometer to pin 3 (non-inverting input) of the op-amp", "Connect the other end of the potentiometer to ground" ], "feedback_loop": [ "Connect a wire from pin 6 (output) directly to pin 2 (inverting input)" ], "load_resistor_conne